In [151]:
import pandas as pd
from reportlab.lib import colors
from reportlab.lib.pagesizes import letter
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
import re

In [152]:
# Parameters
mainpath = r'C:\Users\ChaakHei\Downloads'
excel_file = mainpath+r'\reports.xlsx'  # Excel file which stores the text of the report
sheet_name = '6055'      # Change sheet name as the stock you want to analyze
version = 4             # Change to the desired version number
output_pdf = mainpath+fr'\{sheet_name}.pdf'
stock = sheet_name.zfill(4)
image1_path = mainpath+fr'\{stock}.HK_price_volume.png' 
image2_path = mainpath+fr'\{stock}.HK_financials_line.png'
image3_path = mainpath+fr'\{stock}.HK_financials.png'
image4_path = mainpath+fr'\{stock}.HK_financials_change.png'

In [153]:
# Load the Excel sheet
df = pd.read_excel(excel_file, sheet_name=sheet_name)

# Extract the relevant cell
text = df.iloc[0, version-1] 

In [154]:
# Function to convert simple Markdown to HTML-like format
def convert_markdown_to_paragraphs(markdown_text):
    lines = markdown_text.split('\n')
    paragraphs = []
    styles = getSampleStyleSheet()
    
    # Regular expression to match numbered or bulleted lists with bold text
    list_pattern = re.compile(r'^\s*(\d+\.|[a-z]\.|[-])\s+\*\*(.*?)\*\*:\s*(.*)$')

    for line in lines:
        line = line.strip()
        if line.startswith('# '):
            paragraphs.append(Paragraph(line[2:], styles['Heading1']))
        elif line.startswith('## '):
            paragraphs.append(Paragraph(line[3:], styles['Heading2']))
        elif line.startswith('### '):
            paragraphs.append(Paragraph(line[4:], styles['Heading3']))  # Handle Heading 3
        elif line.startswith('#### '):
            paragraphs.append(Paragraph(line[5:], styles['Heading4']))  # Handle Heading 4
        else:
            match = list_pattern.match(line)
            if match:
                # Construct the list item with bold formatting
                prefix = match.group(1)  # e.g., '1.', 'a.', '-'
                bold_text = match.group(2)  # Text that should be bold
                content = f"{prefix}<b>{bold_text}</b>: {match.group(3)}"
                paragraphs.append(Paragraph(content, styles['BodyText']))
            else:
                paragraphs.append(Paragraph(line, styles['BodyText']))

        paragraphs.append(Spacer(1, 12))  # Add space between paragraphs
    
    return paragraphs

# Convert Markdown text to reportlab paragraphs
story = convert_markdown_to_paragraphs(text)

In [155]:
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Image
from reportlab.lib.pagesizes import letter
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib import utils

In [156]:
# Function to resize images
def resize_image(image_path, max_width, max_height):
    img = utils.ImageReader(image_path)
    width, height = img.getSize()
    
    aspect = width / height
    if aspect > 1:  # Image is wider than tall
        new_width = max_width
        new_height = max_width / aspect
    else:  # Image is taller than wide
        new_height = max_height
        new_width = max_height * aspect
    
    return Image(image_path, new_width, new_height)

In [157]:
# Resize and add images to the report, if the images is not exist then ignore it
try:
    story.append(resize_image(image1_path, 500, 500))
    story.append(Spacer(1, 12))  # Add space between images
except:
    pass

try:
    story.append(resize_image(image2_path, 500, 500))
    story.append(Spacer(1, 12))  # Add space between images
except:
    pass

try:
    story.append(resize_image(image3_path, 500, 500))
    story.append(Spacer(1, 12))  # Add space between images
except:
    pass

try:
    story.append(resize_image(image4_path, 500, 500))
    story.append(Spacer(1, 12))  # Add space between images
except:
    pass

# Create PDF document
pdf = SimpleDocTemplate(output_pdf, pagesize=letter)
pdf.build(story)